<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
See LICENSE file in the root directory for full license text.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 7: Real-World AV Failure Case Studies

**Module 02: Failure Modes and Edge Cases**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/master/02_Failure_Analysis/notebooks/07_AV_Failure_Case_Studies.ipynb)

**Author:** Milin Patel 

---

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the root causes of major AV accidents
- ✅ Analyze the Uber ATG Tempe crash in detail
- ✅ Learn failure taxonomy for perception systems
- ✅ Apply root cause analysis frameworks
- ✅ Extract safety lessons from real incidents
- ✅ Connect failures to system design principles

---

## Setup and Imports

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, Image as IPImage
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

---

## Why Study AV Failures?

### The Importance of Failure Analysis

**"Those who cannot remember the past are condemned to repeat it."** - George Santayana

Studying real-world AV failures is critical because:

1. **Learn from mistakes:** Understand what went wrong to prevent recurrence
2. **Identify systematic issues:** Discover common failure patterns
3. **Improve designs:** Inform better system architectures
4. **Update standards:** Drive safety regulations and best practices
5. **Calibrate expectations:** Understand current technology limitations

### Historical Context

Aviation safety improved dramatically through rigorous accident investigation:
- **1950s:** ~40 fatal accidents per million flights
- **2020s:** <1 fatal accident per 10 million flights

**Key:** Mandatory reporting, independent investigation, industry-wide learning

**Challenge for AVs:** Industry is nascent, less regulated, competitive pressures

In [ ]:
# Visualize major AV incidents timeline
av_incidents = pd.DataFrame({
    'Date': ['2016-05-07', '2016-07-01', '2018-03-18', '2018-03-23', '2019-12-29', '2021-08-16', '2023-10-02'],
    'Company': ['Tesla', 'Tesla', 'Uber ATG', 'Tesla', 'Tesla', 'Tesla', 'Cruise'],
    'Location': ['Florida, USA', 'Pennsylvania, USA', 'Tempe, AZ', 'Mountain View, CA', 'Indiana, USA', 'Texas, USA', 'San Francisco, CA'],
    'Fatality': [True, True, True, True, False, True, False],
    'Description': [
        'Autopilot failed to detect white truck against bright sky',
        'Autopilot collision with truck',
        'Pedestrian struck and killed crossing street',
        'Autopilot crash into highway barrier',
        'Autopilot collision with parked fire truck',
        'Autopilot crash with no driver present',
        'Pedestrian dragged after being hit by human driver'
    ],
    'SAE_Level': [2, 2, 4, 2, 2, 2, 4],
    'Primary_Cause': [
        'Perception failure (camera)',
        'Perception failure (camera)',
        'Perception + safety driver inattention',
        'Perception + driver inattention',
        'Perception failure',
        'Misuse (no driver)',
        'Decision-making failure'
    ]
})

# Convert date to datetime
av_incidents['Date'] = pd.to_datetime(av_incidents['Date'])

# Display table
display(av_incidents[['Date', 'Company', 'Location', 'Fatality', 'SAE_Level']])

print("\n⚠️ Key Observation: Multiple perception failures across different companies!")

In [ ]:
# Visualize incidents timeline
fig, ax = plt.subplots(figsize=(16, 6))

# Plot incidents
for idx, row in av_incidents.iterrows():
    color = 'red' if row['Fatality'] else 'orange'
    marker = 'X' if row['Fatality'] else 'o'
    size = 300 if row['Fatality'] else 150
    
    ax.scatter(row['Date'], row['SAE_Level'], c=color, marker=marker, s=size, 
               alpha=0.7, edgecolors='black', linewidth=2, label=row['Company'] if idx < 2 else '')
    
    # Add label
    ax.text(row['Date'], row['SAE_Level'] + 0.15, row['Company'], 
            ha='center', fontsize=9, fontweight='bold')

ax.set_xlabel('Date', fontsize=14, fontweight='bold')
ax.set_ylabel('SAE Automation Level', fontsize=14, fontweight='bold')
ax.set_title('Timeline of Major Autonomous Vehicle Incidents (2016-2023)', 
             fontsize=16, fontweight='bold')
ax.set_yticks([0, 1, 2, 3, 4, 5])
ax.set_ylim(-0.5, 5.5)
ax.grid(True, alpha=0.3)

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='X', color='w', markerfacecolor='red', markersize=12, label='Fatal'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label='Non-fatal')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=12)

plt.tight_layout()
plt.show()

print("\n📊 Notice: Incidents occur at both Level 2 (driver assistance) and Level 4 (high automation)")

---

## Case Study 1: Uber ATG Tempe Crash (2018)

### Incident Overview

**Date:** March 18, 2018, 9:58 PM 
**Location:** Tempe, Arizona, USA 
**Vehicle:** Volvo XC90 with Uber Advanced Technologies Group (ATG) self-driving system 
**SAE Level:** 4 (High Automation) - testing with safety driver 
**Outcome:** Pedestrian fatality (Elaine Herzberg, 49 years old)

**First pedestrian fatality involving an autonomous vehicle in the United States.**

### What Happened?

1. **Pedestrian:** Walking bicycle across multi-lane road at night (not at crosswalk)
2. **Vehicle:** Traveling at 39 mph in 35 mph zone, autonomous mode engaged
3. **Detection:** System detected pedestrian 5.6 seconds before impact
4. **Classification failures:** System fluctuated between "vehicle," "bicycle," "unknown"
5. **No braking:** Emergency braking was disabled during autonomous operation
6. **Safety driver:** Looking at phone, did not intervene until after impact
7. **Impact:** Vehicle struck pedestrian, who died from injuries

### NTSB Investigation Findings

In [ ]:
# Timeline of Uber crash detection and classification
uber_timeline = pd.DataFrame({
    'Time_Before_Impact': [5.6, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.2, 0.0],
    'Classification': ['Unknown', 'Vehicle', 'Unknown', 'Bicycle', 'Unknown', 'Bicycle', 
                      'Bicycle', 'Bicycle', 'Bicycle', 'Bicycle', 'Impact'],
    'Action_Required': ['Monitor', 'Monitor', 'Monitor', 'Monitor', 'Monitor', 'Alert',
                       'Brake', 'Hard brake', 'Hard brake', 'Emergency brake', 'Too late']
})

display(uber_timeline)

print("\n🚨 Critical Issue: Classification uncertainty delayed decision-making!")
print("   System reset path prediction with each classification change.")

In [ ]:
# Visualize detection timeline
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Classification over time
time_points = uber_timeline['Time_Before_Impact'].values
classifications = uber_timeline['Classification'].values

# Map classifications to numeric values for plotting
class_map = {'Unknown': 0, 'Vehicle': 1, 'Bicycle': 2, 'Impact': 3}
class_numeric = [class_map[c] for c in classifications]

ax1.plot(time_points, class_numeric, marker='o', markersize=10, linewidth=2, color='darkblue')
ax1.set_xlabel('Time Before Impact (seconds)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Classification', fontsize=12, fontweight='bold')
ax1.set_yticks([0, 1, 2, 3])
ax1.set_yticklabels(['Unknown', 'Vehicle', 'Bicycle', 'Impact'])
ax1.set_title('Uber ATG System Classification Over Time', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axvspan(1.2, 5.6, alpha=0.2, color='red', label='Should have braked')
ax1.legend()
ax1.invert_xaxis()

# Required action over time
actions = uber_timeline['Action_Required'].values
action_colors = ['green', 'green', 'green', 'green', 'green', 'yellow', 
                'orange', 'red', 'red', 'darkred', 'black']

ax2.scatter(time_points, range(len(time_points)), c=action_colors, s=200, alpha=0.7)
for i, (t, action) in enumerate(zip(time_points, actions)):
    ax2.text(t, i, action, ha='right', va='center', fontsize=10, fontweight='bold')

ax2.set_xlabel('Time Before Impact (seconds)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Timeline', fontsize=12, fontweight='bold')
ax2.set_title('Required Actions Timeline', fontsize=14, fontweight='bold')
ax2.set_yticks([])
ax2.grid(True, alpha=0.3, axis='x')
ax2.invert_xaxis()

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: System had 5.6 seconds to react but classification uncertainty prevented timely action!")

### Root Causes (NTSB Findings)

**1. Perception System Failures:**
- ❌ Object classification instability (unknown → vehicle → bicycle)
- ❌ Pedestrian crossing mid-block not in expected "crosswalk" scenario
- ❌ System reset trajectory prediction with each classification change
- ❌ Inadequate testing of jaywalking scenarios

**2. System Design Issues:**
- ❌ Emergency braking **disabled** during autonomous mode
- ❌ Reliance on safety driver for emergency intervention (but no monitoring)
- ❌ No redundant safety systems
- ❌ Insufficient sensor coverage for low-light conditions

**3. Organizational/Human Factors:**
- ❌ Safety driver looking at phone (streaming video)
- ❌ Inadequate safety driver monitoring systems
- ❌ Pressure to reduce interventions ("disengagements")
- ❌ Insufficient safety culture and oversight

**4. Regulatory Gaps:**
- ❌ No federal safety standards for autonomous vehicles
- ❌ Limited state-level oversight
- ❌ No mandatory incident reporting

### Lessons Learned

**Design Principles:**
1. **Never disable safety-critical functions** (e.g., emergency braking)
2. **Redundancy:** Multiple independent safety layers
3. **Graceful degradation:** Fail safely when uncertain
4. **Test edge cases:** Jaywalking, unexpected pedestrian behavior

**Organizational:**
1. **Safety culture:** Prioritize safety over development speed
2. **Monitor safety drivers:** Ensure attention and readiness
3. **Transparent reporting:** Share learnings across industry

**Technical:**
1. **Stable classification:** Don't reset decision-making with each class change
2. **Conservative behavior:** Slow down when uncertain
3. **Multi-modal sensing:** Camera + LiDAR + radar fusion

---

## Case Study 2: Tesla Autopilot Incidents

### Florida Crash (May 2016) - First Fatal Autopilot Crash

**Scenario:** Tesla Model S on highway, Autopilot engaged 
**Obstacle:** Tractor-trailer turning left across highway 
**Condition:** Bright sunlight, white truck against bright sky 
**Outcome:** Vehicle drove under truck, driver fatality

**Root Cause:**
- Camera-based perception failed to detect white truck against bright sky (high contrast)
- Radar detected truck but classified as overhead sign (false positive mitigation)
- Driver over-reliance on Autopilot (watching movie)

**Technical Issue:** Camera saturation + sensor fusion logic error

### Mountain View Crash (March 2018)

**Scenario:** Tesla Model X on Highway 101, Autopilot engaged 
**Obstacle:** Concrete highway barrier (lane divider) 
**Condition:** Daylight, clear weather, damaged lane markings 
**Outcome:** Vehicle accelerated into barrier, driver fatality

**Root Cause:**
- Lane-keeping system followed faded/damaged lane markings
- Driver distracted (playing mobile game)
- System did not detect barrier as obstacle

**Technical Issue:** Over-reliance on lane markings, insufficient object detection

### Common Patterns Across Tesla Incidents

1. **Perception limitations:**
 - Camera-only system vulnerable to lighting conditions
 - Difficulty detecting stationary objects (trucks, barriers)
 - Lane marking dependency

2. **Human factors:**
 - Driver misunderstanding of system capabilities (Level 2 ≠ Level 4!)
 - Over-reliance and complacency
 - Reduced monitoring over time

3. **Design trade-offs:**
 - Cost: No LiDAR (Tesla's camera-only approach)
 - User experience: Hands-on-wheel enforcement initially lax
 - False positive reduction → missed true positives

In [ ]:
# Comparison of sensor limitations
sensor_limitations = pd.DataFrame({
    'Scenario': [
        'White truck, bright sky',
        'Stationary barrier',
        'Night, dark clothing',
        'Rain, fog',
        'Direct sunlight glare'
    ],
    'Camera_Only': ['❌ Fails', '⚠️ May miss', '❌ Fails', '⚠️ Degraded', '❌ Fails'],
    'Camera_LiDAR': ['✅ LiDAR detects', '✅ Both detect', '✅ LiDAR detects', '⚠️ LiDAR degraded', '✅ LiDAR works'],
    'Camera_LiDAR_Radar': ['✅✅ Redundant', '✅✅ Redundant', '✅✅ Redundant', '✅ Radar works', '✅✅ Redundant']
})

display(sensor_limitations)

print("\n💡 Lesson: Sensor redundancy critical for safety!")
print("   Camera-only systems have fundamental limitations.")

---

## Case Study 3: Cruise SF Incident (October 2023)

### Incident Overview

**Date:** October 2, 2023 
**Location:** San Francisco, California 
**Vehicle:** Cruise robotaxi (Level 4, no driver) 
**Outcome:** Pedestrian injured and dragged by vehicle

### What Happened?

1. **Initial impact:** Human-driven vehicle struck pedestrian, throwing them into Cruise vehicle's path
2. **Cruise response:** Autonomous vehicle detected impact and **braked**
3. **Problem:** Vehicle then attempted to **pull over** (normal protocol)
4. **Tragedy:** Pedestrian was trapped under vehicle and **dragged 20 feet**
5. **Outcome:** Pedestrian severely injured

### Root Causes

**1. Decision-making failure:**
- ❌ System did not detect pedestrian **under** the vehicle
- ❌ "Pull over after collision" protocol inappropriate for this scenario
- ❌ No sensor coverage for objects under vehicle

**2. Communication failure:**
- ❌ Cruise initially did not disclose dragging to regulators
- ❌ Showed video of initial impact but not subsequent dragging
- ❌ Damaged trust with California DMV and public

**3. Safety process gaps:**
- ❌ Insufficient testing of "secondary collision" scenarios
- ❌ Edge case not covered in safety validation
- ❌ No fail-safe for unknown object under vehicle

### Regulatory Consequence

- California DMV **suspended** Cruise's robotaxi operations
- Cruise **recalled** all vehicles for software update
- Industry-wide scrutiny of Level 4 deployment safety

### Lessons Learned

1. **Expect the unexpected:** Even low-probability scenarios can occur
2. **Complete sensor coverage:** Including under-vehicle sensors
3. **Context-aware behavior:** "Pull over" not always safe action
4. **Transparency:** Full disclosure critical for trust and learning
5. **Testing scope:** Include multi-stage incidents

---

## Failure Taxonomy for AV Perception Systems

A systematic classification of how perception systems can fail.

In [ ]:
# Create failure taxonomy
failure_taxonomy = {
    'Detection Failures': {
        'False Negative (Miss)': 'Object exists but not detected (e.g., Uber pedestrian initially)',
        'False Positive (Phantom)': 'Detection of non-existent object (e.g., radar overhead sign)',
        'Late Detection': 'Object detected but too late for safe response'
    },
    'Classification Failures': {
        'Misclassification': 'Object detected but wrong class (e.g., bicycle as vehicle)',
        'Classification Instability': 'Fluctuating class labels (e.g., Uber unknown→vehicle→bicycle)',
        'Unknown Class': 'Novel object not in training data'
    },
    'Tracking Failures': {
        'Lost Track': 'Object disappears from tracking (occlusion, sensor limit)',
        'ID Switch': 'Object IDs swapped between similar objects',
        'Trajectory Error': 'Incorrect prediction of object motion'
    },
    'Localization Failures': {
        'Position Error': 'Incorrect spatial location of object or vehicle',
        'Map Mismatch': 'HD map outdated or incorrect',
        'GPS Failure': 'Lost or degraded positioning signal'
    },
    'Sensor Failures': {
        'Hardware Fault': 'Camera, LiDAR, radar malfunction',
        'Calibration Drift': 'Sensors misaligned over time',
        'Environmental Degradation': 'Rain, fog, sun glare, darkness',
        'Occlusion': 'Sensor view blocked (dirt, obstruction)'
    },
    'Fusion Failures': {
        'Conflicting Sensors': 'Camera says "car", LiDAR says "empty"',
        'Sensor Timeout': 'Sensor data arrives late, desynchronized',
        'Fusion Logic Error': 'Incorrect weighting or combination'
    },
    'System-Level Failures': {
        'Computational Overload': 'Processing cannot keep up with sensor data rate',
        'Software Bug': 'Code error causes incorrect behavior',
        'Inadequate ODD': 'Operating outside design domain'
    }
}

# Display as structured data
print("📊 AV PERCEPTION FAILURE TAXONOMY\n" + "="*60)
for category, failures in failure_taxonomy.items():
    print(f"\n{category.upper()}")
    print("-" * 60)
    for failure_type, description in failures.items():
        print(f"  • {failure_type}: {description}")

In [ ]:
# Visualize failure taxonomy as tree
from matplotlib.patches import FancyBboxPatch
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(16, 10))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Root
root_box = FancyBboxPatch((3.5, 8.5), 3, 0.8, boxstyle="round,pad=0.1", 
                          facecolor='#ff6b6b', edgecolor='black', linewidth=2)
ax.add_patch(root_box)
ax.text(5, 8.9, 'PERCEPTION FAILURES', ha='center', va='center', 
        fontsize=14, fontweight='bold', color='white')

# Categories
categories = list(failure_taxonomy.keys())
colors = ['#4ecdc4', '#45b7d1', '#96ceb4', '#feca57', '#ff6b6b', '#a29bfe', '#fd79a8']
x_positions = [0.5, 2, 3.5, 5, 6.5, 8, 9.5]
y_position = 6.5

for i, (category, color) in enumerate(zip(categories, colors)):
    x = x_positions[i] if i < len(x_positions) else 0.5 + (i % 7) * 1.5
    y = y_position if i < 7 else y_position - 2
    
    # Draw box
    box = FancyBboxPatch((x - 0.4, y), 0.8, 0.6, boxstyle="round,pad=0.05",
                         facecolor=color, edgecolor='black', linewidth=1.5, alpha=0.7)
    ax.add_patch(box)
    
    # Add text
    text = category.replace(' Failures', '').replace(' ', '\n')
    ax.text(x, y + 0.3, text, ha='center', va='center', 
            fontsize=8, fontweight='bold')
    
    # Draw connection to root
    ax.plot([5, x], [8.5, y + 0.6], 'k-', linewidth=1, alpha=0.5)

# Add title
ax.text(5, 9.7, 'Failure Taxonomy for AV Perception Systems', 
        ha='center', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Use this taxonomy to systematically analyze any AV perception failure!")

---

## Root Cause Analysis Framework

### The "5 Whys" Method

**Example: Uber Crash**

1. **Why did the accident occur?** 
 → Vehicle did not brake for pedestrian

2. **Why didn't it brake?** 
 → Perception system didn't reliably classify the pedestrian

3. **Why didn't it classify reliably?** 
 → System reset trajectory prediction with each classification change

4. **Why did it reset predictions?** 
 → Software design flaw: coupled classification and planning

5. **Why was this design flaw present?** 
 → Insufficient testing of edge cases (jaywalking scenarios)

**Root cause:** Inadequate testing processes and software architecture

### Fishbone (Ishikawa) Diagram

Categorize contributing factors:
- **People:** Safety driver inattention
- **Process:** Insufficient testing, weak safety culture
- **Technology:** Classification instability, disabled emergency brake
- **Environment:** Night, jaywalking (not at crosswalk)
- **Management:** Pressure to reduce interventions

In [ ]:
# Root cause analysis example for Uber crash
root_cause_factors = pd.DataFrame({
    'Category': ['Technical', 'Technical', 'Technical', 'Process', 'Process', 
                 'Human', 'Human', 'Organizational', 'Environmental'],
    'Contributing_Factor': [
        'Classification instability',
        'Emergency braking disabled',
        'Path prediction reset on class change',
        'Inadequate edge case testing',
        'No jaywalking scenarios tested',
        'Safety driver distraction (phone)',
        'No driver monitoring system',
        'Pressure to reduce interventions',
        'Night operation, no crosswalk'
    ],
    'Severity': ['Critical', 'Critical', 'High', 'High', 'High', 
                 'Critical', 'High', 'Medium', 'Medium'],
    'Mitigation': [
        'Stable multi-frame classification',
        'Never disable safety-critical systems',
        'Decouple classification from planning',
        'Expand test scenario library',
        'Systematic ODD boundary testing',
        'Driver monitoring system (DMS)',
        'Mandatory DMS with alerts',
        'Safety-first culture, independent oversight',
        'Improved night perception, assume jaywalking'
    ]
})

display(root_cause_factors)

print("\n✅ Each contributing factor has specific mitigation!")
print("   Systematic analysis → Systematic improvements")

---

## Common Patterns Across AV Failures

### Technical Patterns

1. **Sensor limitations:**
 - Camera: Lighting (glare, darkness), weather, contrast
 - LiDAR: Rain, snow, fog (though better than camera)
 - Radar: Low resolution, ground reflections

2. **Perception challenges:**
 - Stationary objects (trucks, barriers)
 - Unusual poses or orientations
 - Occlusions (partial visibility)
 - Out-of-distribution objects

3. **System design issues:**
 - Insufficient redundancy
 - Single-point failures
 - Inadequate ODD definition
 - Edge case coverage gaps

### Human Factors

1. **Over-reliance:**
 - Misunderstanding system capabilities
 - Complacency over time
 - Attention degradation

2. **Mode confusion:**
 - Not knowing which mode is active
 - Level 2 treated as Level 4
 - Unclear handover protocols

### Organizational Issues

1. **Development pressure:**
 - Speed over safety trade-offs
 - Inadequate testing
 - Premature deployment

2. **Safety culture:**
 - Insufficient oversight
 - Lack of transparency
 - No systematic learning from near-misses

In [ ]:
# Frequency of failure types across incidents
failure_frequency = pd.DataFrame({
    'Failure_Type': [
        'Camera perception failure',
        'Driver/operator inattention',
        'Stationary object not detected',
        'Weather/lighting degradation',
        'Classification error',
        'System design flaw',
        'Inadequate testing',
        'Single sensor type (camera only)'
    ],
    'Frequency': [6, 5, 4, 3, 3, 4, 5, 4],
    'Severity': ['Critical', 'Critical', 'Critical', 'High', 'High', 'Critical', 'High', 'High']
})

# Sort by frequency
failure_frequency = failure_frequency.sort_values('Frequency', ascending=True)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
colors = ['darkred' if s == 'Critical' else 'orange' for s in failure_frequency['Severity']]
ax.barh(failure_frequency['Failure_Type'], failure_frequency['Frequency'], color=colors, alpha=0.7)
ax.set_xlabel('Frequency in Major Incidents', fontsize=12, fontweight='bold')
ax.set_title('Common Failure Types Across AV Incidents', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='darkred', alpha=0.7, label='Critical Severity'),
    Patch(facecolor='orange', alpha=0.7, label='High Severity')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

print("\n📊 Top failure patterns:")
print("   1. Camera perception limitations")
print("   2. Human factor (operator inattention)")
print("   3. Inadequate testing and validation")

---

## Exercise: Failure Analysis

**Scenario:** An autonomous shuttle operating on a university campus (Level 4, 25 km/h max) has an incident:

- **Conditions:** Early morning, light fog, wet road
- **Event:** Student on electric scooter crosses in front of shuttle at intersection
- **Result:** Shuttle does not brake, student swerves to avoid collision
- **System logs:** Object detected 2.5 seconds before, classified as "bicycle" with 45% confidence

**Your Task:**

1. **Classify the failure** using the taxonomy above
2. **Apply "5 Whys"** to find root cause
3. **Identify contributing factors** (technical, process, environmental)
4. **Propose mitigations** for each factor
5. **Lessons learned** for system design

In [ ]:
# TODO: Complete your analysis

your_analysis = {
    'failure_classification': '',  # e.g., "Detection Failure: Late Detection"
    'five_whys': [
        # Add your 5 why questions and answers
    ],
    'contributing_factors': {
        'technical': [],
        'process': [],
        'environmental': []
    },
    'mitigations': [
        # Your proposed mitigations
    ],
    'lessons_learned': [
        # Key takeaways for design
    ]
}

# Print your analysis
print("Your Failure Analysis:")
print(f"\nClassification: {your_analysis['failure_classification']}")
print("\n5 Whys:")
for i, why in enumerate(your_analysis['five_whys'], 1):
    print(f"  {i}. {why}")

print("\n🤔 Reflection:")
print("   - Could this incident have been prevented?")
print("   - What would you change in the system design?")
print("   - How does this relate to ISO 21448 SOTIF?")

---

## Key Takeaways

### Real-World Failures Teach Us
- **Uber ATG (2018):** Classification instability + disabled safety systems + human factors
- **Tesla Autopilot:** Camera limitations + driver over-reliance + edge cases
- **Cruise SF (2023):** Decision-making failure + inadequate sensor coverage

### Failure Taxonomy
- **Seven categories:** Detection, Classification, Tracking, Localization, Sensor, Fusion, System-level
- **Use for systematic analysis** of any perception failure

### Common Patterns
- **Technical:** Camera limitations, stationary objects, lighting/weather
- **Human:** Over-reliance, mode confusion, inattention
- **Organizational:** Speed over safety, inadequate testing

### Root Cause Analysis
- **5 Whys method:** Dig deeper than surface symptoms
- **Fishbone diagram:** Categorize contributing factors
- **Systematic mitigations:** Address each factor

### Design Principles from Failures
1. ✅ **Never disable safety systems**
2. ✅ **Multi-modal sensing** (camera + LiDAR + radar)
3. ✅ **Fail safely** when uncertain
4. ✅ **Comprehensive testing** including edge cases
5. ✅ **Safety culture** over development speed
6. ✅ **Transparent reporting** and learning

---

## Next: Out-of-Distribution Detection

Now that we understand **what** can go wrong, let's learn **how to detect** when the system encounters unknown situations!

**Ready?** Open `08_OOD_Detection.ipynb`

---

*Notebook created by Milin Patel | Hochschule Kempten* 
*Last updated: 2025-01-18*